In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset

pip install 'aif360[LawSchoolGPA]'


In [2]:
raw_df=pd.read_csv("Bank Customer Churn Prediction.csv",index_col=0)
raw_df.drop_duplicates()
display(raw_df)

credit_score  country  gender  age  tenure    balance  \
customer_id                                                          
15634602              619   France  Female   42       2       0.00   
15647311              608    Spain  Female   41       1   83807.86   
15619304              502   France  Female   42       8  159660.80   
15701354              699   France  Female   39       1       0.00   
15737888              850    Spain  Female   43       2  125510.82   
...                   ...      ...     ...  ...     ...        ...   
15606229              771   France    Male   39       5       0.00   
15569892              516   France    Male   35      10   57369.61   
15584532              709   France  Female   36       7       0.00   
15682355              772  Germany    Male   42       3   75075.31   
15628319              792   France  Female   28       4  130142.79   

             products_number  credit_card  active_member  estimated_salary  \
customer_id                                                                  
15634602                   1            1              1         101348.88   
15647311                   1            0              1         112542.58   
15619304                   3            1              0         113931.57   
15701354                   2            0              0          93826.63   
15737888                   1            1              1          79084.10   
...                      ...          ...            ...               ...   
15606229                   2            1              0          96270.64   
15569892                   1            1              1         101699.77   
15584532                   1            0              1          42085.58   
15682355                   2            1              0          92888.52   
15628319                   1            1              0          38190.78   

             churn  
customer_id         
15634602         1  
15647311         0  
15619304         1  
15701354         0  
15737888         0  
...            ...  
15606229         0  
15569892         0  
15584532         1  
15682355         1  
15628319         0  

[10000 rows x 11 columns]

In [3]:
man_df=raw_df.copy().loc[raw_df["gender"]=="Male"].loc[:,['gender','churn']]
display(man_df)
woman_df=raw_df.copy().loc[raw_df["gender"]!="Male"].loc[:,['gender','churn']]
display(woman_df)

gender  churn
customer_id              
15574012      Male      1
15592531      Male      0
15792365      Male      0
15592389      Male      0
15767821      Male      0
...            ...    ...
15657105      Male      0
15569266      Male      0
15606229      Male      0
15569892      Male      0
15682355      Male      1

[5457 rows x 2 columns]

gender  churn
customer_id               
15634602     Female      1
15647311     Female      0
15619304     Female      1
15701354     Female      0
15737888     Female      0
...             ...    ...
15768163     Female      1
15769959     Female      1
15719294     Female      0
15584532     Female      1
15628319     Female      0

[4543 rows x 2 columns]

In [4]:
print(man_df.shape[0]/raw_df.shape[0])
print(woman_df.shape[0]/raw_df.shape[0])

man_label=man_df["churn"].value_counts()
print(man_label[1]/man_df.shape[0])
print(man_label[0]/man_df.shape[0])

women_label=woman_df["churn"].value_counts()
print(women_label[1]/woman_df.shape[0])
print(women_label[0]/woman_df.shape[0])

0.5457
0.4543
0.16455928165658787
0.8354407183434122
0.2507153863086066
0.7492846136913933


In [5]:
def min_max_normalisation(series):
    return (series - series.min()) / (series.max() - series.min())

df=raw_df.copy(deep=True)
#normalize PreviousSalary feature
df["balance"]=min_max_normalisation(df["balance"])
df["estimated_salary"]=min_max_normalisation(df["estimated_salary"])
df["credit_score"]=min_max_normalisation(df["credit_score"])
df["age"]=min_max_normalisation(df["age"])

display(df)

credit_score  country  gender       age  tenure   balance  \
customer_id                                                              
15634602            0.538   France  Female  0.324324       2  0.000000   
15647311            0.516    Spain  Female  0.310811       1  0.334031   
15619304            0.304   France  Female  0.324324       8  0.636357   
15701354            0.698   France  Female  0.283784       1  0.000000   
15737888            1.000    Spain  Female  0.337838       2  0.500246   
...                   ...      ...     ...       ...     ...       ...   
15606229            0.842   France    Male  0.283784       5  0.000000   
15569892            0.332   France    Male  0.229730      10  0.228657   
15584532            0.718   France  Female  0.243243       7  0.000000   
15682355            0.844  Germany    Male  0.324324       3  0.299226   
15628319            0.884   France  Female  0.135135       4  0.518708   

             products_number  credit_card  active_member  estimated_salary  \
customer_id                                                                  
15634602                   1            1              1          0.506735   
15647311                   1            0              1          0.562709   
15619304                   3            1              0          0.569654   
15701354                   2            0              0          0.469120   
15737888                   1            1              1          0.395400   
...                      ...          ...            ...               ...   
15606229                   2            1              0          0.481341   
15569892                   1            1              1          0.508490   
15584532                   1            0              1          0.210390   
15682355                   2            1              0          0.464429   
15628319                   1            1              0          0.190914   

             churn  
customer_id         
15634602         1  
15647311         0  
15619304         1  
15701354         0  
15737888         0  
...            ...  
15606229         0  
15569892         0  
15584532         1  
15682355         1  
15628319         0  

[10000 rows x 11 columns]

In [6]:
folds=np.array_split(df.sample(frac=1),5)

In [9]:
def get_dataset(input_df):
    dataset=StandardDataset(df=input_df,label_name="churn",favorable_classes=[1],scores_name="",protected_attribute_names=["gender"],
                                    privileged_classes=[["Male"]],categorical_features=['country'])
    return dataset

In [10]:
Cross_Validation_Column=["Cross Validate 1","Cross Validate 2","Cross Validate 3","Cross Validate 4","Cross Validate 5","Average"]
Accuracy_data=pd.DataFrame(columns=Cross_Validation_Column)
DisparateImpact_data=pd.DataFrame(columns=Cross_Validation_Column)
DisparateParity_data=pd.DataFrame(columns=Cross_Validation_Column)
EqualOpportunity_data=pd.DataFrame(columns=Cross_Validation_Column)
EqualizedOdds_data=pd.DataFrame(columns=Cross_Validation_Column)
#Read progress from local file if something happend that the jupyter kernel needs to be restarted (like fix bug from source library and reloaded it)
#merics_data=pd.read_csv("Benchmark_data.csv",index_col=0)

privileged_groups = [{"gender":1}]
unprivileged_groups = [{"gender":0}]

In [11]:
def cross_validation(row_name,train_test_process):
    global Accuracy_data
    global DisparateImpact_data
    global DisparateParity_data
    global EqualOpportunity_data
    global EqualizedOdds_data
    accuracy_row=[]
    disparateImpcat_row=[]
    disparateParity_row=[]
    EqualOpportunity_row=[]
    EqualizedOdss_row=[]
    for fold_index in range(5):
        test_fold=folds[fold_index]
        train_set=get_dataset(pd.concat([f for f in folds if f is not test_fold]))
        test_set=get_dataset(test_fold)
        test_pred=train_test_process(train_set,test_set)
        metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups)
        #Record Result
        accuracy_row.append(metric.accuracy())
        disparateImpcat_row.append(metric.disparate_impact())
        disparateParity_row.append(metric.statistical_parity_difference())
        EqualOpportunity_row.append(metric.true_positive_rate_difference())
        EqualizedOdss_row.append(metric.false_positive_rate_difference())
    accuracy_row.append(np.average(accuracy_row))
    disparateImpcat_row.append(np.average(disparateImpcat_row))
    disparateParity_row.append(np.average(disparateParity_row))
    EqualOpportunity_row.append(np.average(EqualOpportunity_row))
    EqualizedOdss_row.append(np.average(EqualizedOdss_row))
    #Record Cross Validation
    Accuracy_data=pd.concat([Accuracy_data,pd.DataFrame([accuracy_row],index=[row_name],columns=Cross_Validation_Column)])
    Accuracy_data.columns.name="Accuracy"
    display(Accuracy_data)
    DisparateImpact_data=pd.concat([DisparateImpact_data,pd.DataFrame([disparateImpcat_row],index=[row_name],columns=Cross_Validation_Column)])
    DisparateImpact_data.columns.name="Disparate Impact"
    display(DisparateImpact_data)
    DisparateParity_data=pd.concat([DisparateParity_data,pd.DataFrame([disparateParity_row],index=[row_name],columns=Cross_Validation_Column)])
    DisparateParity_data.columns.name="Disparate Parity"
    display(DisparateParity_data)
    EqualOpportunity_data=pd.concat([EqualOpportunity_data,pd.DataFrame([EqualOpportunity_row],index=[row_name],columns=Cross_Validation_Column)])
    EqualOpportunity_data.columns.name="Equal Opportuniy"
    display(EqualOpportunity_data)
    EqualizedOdds_data=pd.concat([EqualizedOdds_data,pd.DataFrame([EqualizedOdss_row],index=[row_name],columns=Cross_Validation_Column)])
    EqualizedOdds_data.columns.name="Equalized Odds"
    display(EqualizedOdds_data)

In [12]:
#Train RandomForest as Baseline model on original Datasets
def baseline_CV(train_set, test_set):
    #Training
    baseline=RandomForestClassifier()
    baseline.fit(train_set.features,train_set.labels.ravel())
    #Testing
    pred=baseline.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("Baseline",baseline_CV)

Accuracy  Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline             0.859            0.8695            0.8555   

Accuracy  Cross Validate 4  Cross Validate 5  Average  
Baseline             0.873             0.857   0.8628

Disparate Impact  Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                  1.561193           1.46609          2.033385   

Disparate Impact  Cross Validate 4  Cross Validate 5   Average  
Baseline                  1.884051          1.503438  1.689631

Disparate Parity  Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                  0.059128          0.047793          0.083322   

Disparate Parity  Cross Validate 4  Cross Validate 5   Average  
Baseline                  0.074396          0.048614  0.062651

Equal Opportuniy  Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                  0.067797          0.029337          0.111285   

Equal Opportuniy  Cross Validate 4  Cross Validate 5   Average  
Baseline                  0.141108         -0.036105  0.062684

Equalized Odds  Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                0.005676           0.00924          0.032212   

Equalized Odds  Cross Validate 4  Cross Validate 5   Average  
Baseline                0.017331          0.021796  0.017251

In [13]:
#Zhange et al. implementation
import tensorflow.compat.v1 as tf
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
def AdversarialDebiase_CV(train_set,test_set):
    #Training
    tf.disable_eager_execution()
    tf.reset_default_graph()
    with tf.Session() as sess:
        debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                                unprivileged_groups = unprivileged_groups,
                                scope_name='debiased_classifier',
                                debias=True,
                                sess=sess)
        debiased_model.fit(train_set)
    #Testing
        test_pred=debiased_model.predict(test_set)
    return test_pred

cross_validation("AdversarialDebiase",AdversarialDebiase_CV)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.709416; batch adversarial loss: 0.662851
epoch 1; iter: 0; batch classifier loss: 0.420827; batch adversarial loss: 0.687737
epoch 2; iter: 0; batch classifier loss: 0.483966; batch adversarial loss: 0.695637
epoch 3; iter: 0; batch classifier loss: 0.431690; batch adversarial loss: 0.695629
epoch 4; iter: 0; batch classifier loss: 0.470558; batch adversarial loss: 0.711989
epoch 5; iter: 0; batch classifier loss: 0.508259; batch adversarial loss: 0.708284
epoch 6; iter: 0; batch classifier loss: 0.401220; batch adversarial loss: 0.687702
epoch 7; iter: 0; batch classifier loss: 0.432514; batch adversarial loss: 0.682461
epoch 8; iter: 0; batch classifier loss: 0.481751; batch adversarial loss: 0.687104
epoch 9; iter: 0; batch classifier loss: 0.578854; batch adversarial loss: 0.675301
epoch 10; iter: 0; batch classifier loss: 0.435119; batch adversarial loss: 0.679118
epoch 11; iter: 0; batch classifier loss: 0.417975; batch adversarial loss:

Accuracy            Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                       0.859            0.8695            0.8555   
AdversarialDebiase             0.849            0.8560            0.8475   

Accuracy            Cross Validate 4  Cross Validate 5  Average  
Baseline                      0.8730            0.8570   0.8628  
AdversarialDebiase            0.8675            0.8475   0.8535

Disparate Impact    Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                    1.561193          1.466090          2.033385   
AdversarialDebiase          1.074928          1.127915          1.716931   

Disparate Impact    Cross Validate 4  Cross Validate 5   Average  
Baseline                    1.884051          1.503438  1.689631  
AdversarialDebiase          0.989561          1.257602  1.233387

Disparate Parity    Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                    0.059128          0.047793          0.083322   
AdversarialDebiase          0.008656          0.015786          0.077075   

Disparate Parity    Cross Validate 4  Cross Validate 5   Average  
Baseline                    0.074396          0.048614  0.062651  
AdversarialDebiase         -0.001159          0.021527  0.024377

Equal Opportuniy    Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                    0.067797          0.029337          0.111285   
AdversarialDebiase         -0.029661         -0.060538          0.059636   

Equal Opportuniy    Cross Validate 4  Cross Validate 5   Average  
Baseline                    0.141108         -0.036105  0.062684  
AdversarialDebiase         -0.043579         -0.064457 -0.027720

Equalized Odds      Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                    0.005676          0.009240          0.032212   
AdversarialDebiase         -0.027007         -0.004956          0.035167   

Equalized Odds      Cross Validate 4  Cross Validate 5   Average  
Baseline                    0.017331          0.021796  0.017251  
AdversarialDebiase         -0.029862          0.003440 -0.004644

In [14]:
#Agarwal et al., 2018, Agarwal et al., 2019 Implementation
from aif360.algorithms.inprocessing import GridSearchReduction
def GridSearchReduction_EO_CV(train_set,test_set):
    #Training
    gsr_eo_model=GridSearchReduction(estimator=RandomForestClassifier(),constraints="EqualizedOdds",prot_attr="gender")
    gsr_eo_model.fit(train_set)
    #Testing
    test_pred=gsr_eo_model.predict(test_set)
    return test_pred

cross_validation("GridSearchReduction(EO)",GridSearchReduction_EO_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\prepro

Accuracy                 Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                            0.859            0.8695            0.8555   
AdversarialDebiase                  0.849            0.8560            0.8475   
GridSearchReduction(EO)             0.857            0.8630            0.8510   

Accuracy                 Cross Validate 4  Cross Validate 5  Average  
Baseline                           0.8730            0.8570   0.8628  
AdversarialDebiase                 0.8675            0.8475   0.8535  
GridSearchReduction(EO)            0.8655            0.8565   0.8586

Disparate Impact         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         1.561193          1.466090          2.033385   
AdversarialDebiase               1.074928          1.127915          1.716931   
GridSearchReduction(EO)          1.356382          1.236195          1.709511   

Disparate Impact         Cross Validate 4  Cross Validate 5   Average  
Baseline                         1.884051          1.503438  1.689631  
AdversarialDebiase               0.989561          1.257602  1.233387  
GridSearchReduction(EO)          1.462300          1.233797  1.399637

Disparate Parity         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         0.059128          0.047793          0.083322   
AdversarialDebiase               0.008656          0.015786          0.077075   
GridSearchReduction(EO)          0.041501          0.029149          0.064441   

Disparate Parity         Cross Validate 4  Cross Validate 5   Average  
Baseline                         0.074396          0.048614  0.062651  
AdversarialDebiase              -0.001159          0.021527  0.024377  
GridSearchReduction(EO)          0.045113          0.026484  0.041338

Equal Opportuniy         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         0.067797          0.029337          0.111285   
AdversarialDebiase              -0.029661         -0.060538          0.059636   
GridSearchReduction(EO)          0.022599         -0.041232          0.057994   

Equal Opportuniy         Cross Validate 4  Cross Validate 5   Average  
Baseline                         0.141108         -0.036105  0.062684  
AdversarialDebiase              -0.043579         -0.064457 -0.027720  
GridSearchReduction(EO)          0.054446         -0.098174 -0.000874

Equalized Odds           Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         0.005676          0.009240          0.032212   
AdversarialDebiase              -0.027007         -0.004956          0.035167   
GridSearchReduction(EO)         -0.004992          0.003874          0.023028   

Equalized Odds           Cross Validate 4  Cross Validate 5   Average  
Baseline                         0.017331          0.021796  0.017251  
AdversarialDebiase              -0.029862          0.003440 -0.004644  
GridSearchReduction(EO)          0.003750          0.008810  0.006894

In [15]:
#Agarwal et al., 2018, Agarwal et al., 2019 Implementation

def GridSearchReduction_DP_CV(train_set,test_set):
    #Training
    gsr_dp_model=GridSearchReduction(estimator=RandomForestClassifier(),constraints="DemographicParity",prot_attr="gender")
    gsr_dp_model.fit(train_set)
    #Testing
    test_pred=gsr_dp_model.predict(test_set)
    return test_pred

cross_validation("GridSearchReduction(DP)",GridSearchReduction_DP_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\prepro

Accuracy                 Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                           0.8590            0.8695            0.8555   
AdversarialDebiase                 0.8490            0.8560            0.8475   
GridSearchReduction(EO)            0.8570            0.8630            0.8510   
GridSearchReduction(DP)            0.8595            0.8680            0.8530   

Accuracy                 Cross Validate 4  Cross Validate 5  Average  
Baseline                           0.8730            0.8570   0.8628  
AdversarialDebiase                 0.8675            0.8475   0.8535  
GridSearchReduction(EO)            0.8655            0.8565   0.8586  
GridSearchReduction(DP)            0.8710            0.8610   0.8625

Disparate Impact         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         1.561193          1.466090          2.033385   
AdversarialDebiase               1.074928          1.127915          1.716931   
GridSearchReduction(EO)          1.356382          1.236195          1.709511   
GridSearchReduction(DP)          1.408630          1.255491          1.642589   

Disparate Impact         Cross Validate 4  Cross Validate 5   Average  
Baseline                         1.884051          1.503438  1.689631  
AdversarialDebiase               0.989561          1.257602  1.233387  
GridSearchReduction(EO)          1.462300          1.233797  1.399637  
GridSearchReduction(DP)          1.335284          1.148326  1.358064

Disparate Parity         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         0.059128          0.047793          0.083322   
AdversarialDebiase               0.008656          0.015786          0.077075   
GridSearchReduction(EO)          0.041501          0.029149          0.064441   
GridSearchReduction(DP)          0.046452          0.030140          0.057768   

Disparate Parity         Cross Validate 4  Cross Validate 5   Average  
Baseline                         0.074396          0.048614  0.062651  
AdversarialDebiase              -0.001159          0.021527  0.024377  
GridSearchReduction(EO)          0.045113          0.026484  0.041338  
GridSearchReduction(DP)          0.032418          0.017353  0.036826

Equal Opportuniy         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         0.067797          0.029337          0.111285   
AdversarialDebiase              -0.029661         -0.060538          0.059636   
GridSearchReduction(EO)          0.022599         -0.041232          0.057994   
GridSearchReduction(DP)          0.001412         -0.021737          0.049336   

Equal Opportuniy         Cross Validate 4  Cross Validate 5   Average  
Baseline                         0.141108         -0.036105  0.062684  
AdversarialDebiase              -0.043579         -0.064457 -0.027720  
GridSearchReduction(EO)          0.054446         -0.098174 -0.000874  
GridSearchReduction(DP)          0.039056         -0.120274 -0.010441

Equalized Odds           Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         0.005676          0.009240          0.032212   
AdversarialDebiase              -0.027007         -0.004956          0.035167   
GridSearchReduction(EO)         -0.004992          0.003874          0.023028   
GridSearchReduction(DP)          0.006037         -0.000629          0.016887   

Equalized Odds           Cross Validate 4  Cross Validate 5   Average  
Baseline                         0.017331          0.021796  0.017251  
AdversarialDebiase              -0.029862          0.003440 -0.004644  
GridSearchReduction(EO)          0.003750          0.008810  0.006894  
GridSearchReduction(DP)         -0.008896          0.001457  0.002971

In [16]:
#Kamiran and Calders, 2012 implementation
from aif360.algorithms.preprocessing import Reweighing
def Reweight_CV(train_set,test_set):
    #Training
    reweight=Reweighing(privileged_groups=privileged_groups,unprivileged_groups=unprivileged_groups)
    reweight.fit(train_set)
    reweight_set=reweight.transform(train_set.copy(deepcopy=True))
    reweight_test = reweight.transform(test_set.copy(deepcopy=True))

    reweight_model=RandomForestClassifier()
    reweight_model.fit(reweight_set.features,reweight_set.labels.ravel(),sample_weight=train_set.instance_weights)
    #Testing
    pred=reweight_model.predict(reweight_test.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("Reweight",Reweight_CV)

Accuracy                 Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                           0.8590            0.8695            0.8555   
AdversarialDebiase                 0.8490            0.8560            0.8475   
GridSearchReduction(EO)            0.8570            0.8630            0.8510   
GridSearchReduction(DP)            0.8595            0.8680            0.8530   
Reweight                           0.8575            0.8725            0.8585   

Accuracy                 Cross Validate 4  Cross Validate 5  Average  
Baseline                           0.8730            0.8570   0.8628  
AdversarialDebiase                 0.8675            0.8475   0.8535  
GridSearchReduction(EO)            0.8655            0.8565   0.8586  
GridSearchReduction(DP)            0.8710            0.8610   0.8625  
Reweight                           0.8710            0.8550   0.8629

Disparate Impact         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         1.561193          1.466090          2.033385   
AdversarialDebiase               1.074928          1.127915          1.716931   
GridSearchReduction(EO)          1.356382          1.236195          1.709511   
GridSearchReduction(DP)          1.408630          1.255491          1.642589   
Reweight                         1.724320          1.555878          2.016160   

Disparate Impact         Cross Validate 4  Cross Validate 5   Average  
Baseline                         1.884051          1.503438  1.689631  
AdversarialDebiase               0.989561          1.257602  1.233387  
GridSearchReduction(EO)          1.462300          1.233797  1.399637  
GridSearchReduction(DP)          1.335284          1.148326  1.358064  
Reweight                         1.945116          1.541119  1.756519

Disparate Parity         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         0.059128          0.047793          0.083322   
AdversarialDebiase               0.008656          0.015786          0.077075   
GridSearchReduction(EO)          0.041501          0.029149          0.064441   
GridSearchReduction(DP)          0.046452          0.030140          0.057768   
Reweight                         0.072298          0.056496          0.080991   

Disparate Parity         Cross Validate 4  Cross Validate 5   Average  
Baseline                         0.074396          0.048614  0.062651  
AdversarialDebiase              -0.001159          0.021527  0.024377  
GridSearchReduction(EO)          0.045113          0.026484  0.041338  
GridSearchReduction(DP)          0.032418          0.017353  0.036826  
Reweight                         0.078689          0.053258  0.068346

Equal Opportuniy         Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         0.067797          0.029337          0.111285   
AdversarialDebiase              -0.029661         -0.060538          0.059636   
GridSearchReduction(EO)          0.022599         -0.041232          0.057994   
GridSearchReduction(DP)          0.001412         -0.021737          0.049336   
Reweight                         0.103107          0.007505          0.115614   

Equal Opportuniy         Cross Validate 4  Cross Validate 5   Average  
Baseline                         0.141108         -0.036105  0.062684  
AdversarialDebiase              -0.043579         -0.064457 -0.027720  
GridSearchReduction(EO)          0.054446         -0.098174 -0.000874  
GridSearchReduction(DP)          0.039056         -0.120274 -0.010441  
Reweight                         0.136253         -0.018234  0.068849

Equalized Odds           Cross Validate 1  Cross Validate 2  Cross Validate 3  \
Baseline                         0.005676          0.009240          0.032212   
AdversarialDebiase              -0.027007         -0.004956          0.035167   
GridSearchReduction(EO)         -0.004992          0.003874          0.023028   
GridSearchReduction(DP)          0.006037         -0.000629          0.016887   
Reweight                         0.013751          0.024708          0.027520   

Equalized Odds           Cross Validate 4  Cross Validate 5   Average  
Baseline                         0.017331          0.021796  0.017251  
AdversarialDebiase              -0.029862          0.003440 -0.004644  
GridSearchReduction(EO)          0.003750          0.008810  0.006894  
GridSearchReduction(DP)         -0.008896          0.001457  0.002971  
Reweight                         0.024302          0.022860  0.022628

In [17]:
#Feldman et al., 2015 implementation
from aif360.algorithms.preprocessing import DisparateImpactRemover

def DisparateImpactRemover_05_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=0.5,sensitive_attribute="gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(0.5)",DisparateImpactRemover_05_CV)

Accuracy                     Cross Validate 1  Cross Validate 2  \
Baseline                               0.8590            0.8695   
AdversarialDebiase                     0.8490            0.8560   
GridSearchReduction(EO)                0.8570            0.8630   
GridSearchReduction(DP)                0.8595            0.8680   
Reweight                               0.8575            0.8725   
DisparateImpactRemover(0.5)            0.8590            0.8715   

Accuracy                     Cross Validate 3  Cross Validate 4  \
Baseline                               0.8555            0.8730   
AdversarialDebiase                     0.8475            0.8675   
GridSearchReduction(EO)                0.8510            0.8655   
GridSearchReduction(DP)                0.8530            0.8710   
Reweight                               0.8585            0.8710   
DisparateImpactRemover(0.5)            0.8565            0.8715   

Accuracy                     Cross Validate 5  Average  
Baseline                               0.8570   0.8628  
AdversarialDebiase                     0.8475   0.8535  
GridSearchReduction(EO)                0.8565   0.8586  
GridSearchReduction(DP)                0.8610   0.8625  
Reweight                               0.8550   0.8629  
DisparateImpactRemover(0.5)            0.8535   0.8624

Disparate Impact             Cross Validate 1  Cross Validate 2  \
Baseline                             1.561193          1.466090   
AdversarialDebiase                   1.074928          1.127915   
GridSearchReduction(EO)              1.356382          1.236195   
GridSearchReduction(DP)              1.408630          1.255491   
Reweight                             1.724320          1.555878   
DisparateImpactRemover(0.5)          1.669753          1.396437   

Disparate Impact             Cross Validate 3  Cross Validate 4  \
Baseline                             2.033385          1.884051   
AdversarialDebiase                   1.716931          0.989561   
GridSearchReduction(EO)              1.709511          1.462300   
GridSearchReduction(DP)              1.642589          1.335284   
Reweight                             2.016160          1.945116   
DisparateImpactRemover(0.5)          1.898723          1.825781   

Disparate Impact             Cross Validate 5   Average  
Baseline                             1.503438  1.689631  
AdversarialDebiase                   1.257602  1.233387  
GridSearchReduction(EO)              1.233797  1.399637  
GridSearchReduction(DP)              1.148326  1.358064  
Reweight                             1.541119  1.756519  
DisparateImpactRemover(0.5)          1.461285  1.650396

Disparate Parity             Cross Validate 1  Cross Validate 2  \
Baseline                             0.059128          0.047793   
AdversarialDebiase                   0.008656          0.015786   
GridSearchReduction(EO)              0.041501          0.029149   
GridSearchReduction(DP)              0.046452          0.030140   
Reweight                             0.072298          0.056496   
DisparateImpactRemover(0.5)          0.066852          0.041730   

Disparate Parity             Cross Validate 3  Cross Validate 4  \
Baseline                             0.083322          0.074396   
AdversarialDebiase                   0.077075         -0.001159   
GridSearchReduction(EO)              0.064441          0.045113   
GridSearchReduction(DP)              0.057768          0.032418   
Reweight                             0.080991          0.078689   
DisparateImpactRemover(0.5)          0.072464          0.071711   

Disparate Parity             Cross Validate 5   Average  
Baseline                             0.048614  0.062651  
AdversarialDebiase                   0.021527  0.024377  
GridSearchReduction(EO)              0.026484  0.041338  
GridSearchReduction(DP)              0.017353  0.036826  
Reweight                             0.053258  0.068346  
DisparateImpactRemover(0.5)          0.045829  0.059717

Equal Opportuniy             Cross Validate 1  Cross Validate 2  \
Baseline                             0.067797          0.029337   
AdversarialDebiase                  -0.029661         -0.060538   
GridSearchReduction(EO)              0.022599         -0.041232   
GridSearchReduction(DP)              0.001412         -0.021737   
Reweight                             0.103107          0.007505   
DisparateImpactRemover(0.5)          0.079096         -0.010432   

Equal Opportuniy             Cross Validate 3  Cross Validate 4  \
Baseline                             0.111285          0.141108   
AdversarialDebiase                   0.059636         -0.043579   
GridSearchReduction(EO)              0.057994          0.054446   
GridSearchReduction(DP)              0.049336          0.039056   
Reweight                             0.115614          0.136253   
DisparateImpactRemover(0.5)          0.073817          0.098963   

Equal Opportuniy             Cross Validate 5   Average  
Baseline                            -0.036105  0.062684  
AdversarialDebiase                  -0.064457 -0.027720  
GridSearchReduction(EO)             -0.098174 -0.000874  
GridSearchReduction(DP)             -0.120274 -0.010441  
Reweight                            -0.018234  0.068849  
DisparateImpactRemover(0.5)         -0.044335  0.039422

Equalized Odds               Cross Validate 1  Cross Validate 2  \
Baseline                             0.005676          0.009240   
AdversarialDebiase                  -0.027007         -0.004956   
GridSearchReduction(EO)             -0.004992          0.003874   
GridSearchReduction(DP)              0.006037         -0.000629   
Reweight                             0.013751          0.024708   
DisparateImpactRemover(0.5)          0.013028          0.011433   

Equalized Odds               Cross Validate 3  Cross Validate 4  \
Baseline                             0.032212          0.017331   
AdversarialDebiase                   0.035167         -0.029862   
GridSearchReduction(EO)              0.023028          0.003750   
GridSearchReduction(DP)              0.016887         -0.008896   
Reweight                             0.027520          0.024302   
DisparateImpactRemover(0.5)          0.028625          0.024302   

Equalized Odds               Cross Validate 5   Average  
Baseline                             0.021796  0.017251  
AdversarialDebiase                   0.003440 -0.004644  
GridSearchReduction(EO)              0.008810  0.006894  
GridSearchReduction(DP)              0.001457  0.002971  
Reweight                             0.022860  0.022628  
DisparateImpactRemover(0.5)          0.021389  0.019755

In [18]:
#Feldman et al., 2015 implementation
def DisparateImpactRemover_07_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=0.7,sensitive_attribute="gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(0.7)",DisparateImpactRemover_07_CV)

Accuracy                     Cross Validate 1  Cross Validate 2  \
Baseline                               0.8590            0.8695   
AdversarialDebiase                     0.8490            0.8560   
GridSearchReduction(EO)                0.8570            0.8630   
GridSearchReduction(DP)                0.8595            0.8680   
Reweight                               0.8575            0.8725   
DisparateImpactRemover(0.5)            0.8590            0.8715   
DisparateImpactRemover(0.7)            0.8595            0.8735   

Accuracy                     Cross Validate 3  Cross Validate 4  \
Baseline                               0.8555            0.8730   
AdversarialDebiase                     0.8475            0.8675   
GridSearchReduction(EO)                0.8510            0.8655   
GridSearchReduction(DP)                0.8530            0.8710   
Reweight                               0.8585            0.8710   
DisparateImpactRemover(0.5)            0.8565            0.8715   
DisparateImpactRemover(0.7)            0.8510            0.8715   

Accuracy                     Cross Validate 5  Average  
Baseline                               0.8570   0.8628  
AdversarialDebiase                     0.8475   0.8535  
GridSearchReduction(EO)                0.8565   0.8586  
GridSearchReduction(DP)                0.8610   0.8625  
Reweight                               0.8550   0.8629  
DisparateImpactRemover(0.5)            0.8535   0.8624  
DisparateImpactRemover(0.7)            0.8565   0.8624

Disparate Impact             Cross Validate 1  Cross Validate 2  \
Baseline                             1.561193          1.466090   
AdversarialDebiase                   1.074928          1.127915   
GridSearchReduction(EO)              1.356382          1.236195   
GridSearchReduction(DP)              1.408630          1.255491   
Reweight                             1.724320          1.555878   
DisparateImpactRemover(0.5)          1.669753          1.396437   
DisparateImpactRemover(0.7)          1.550854          1.384249   

Disparate Impact             Cross Validate 3  Cross Validate 4  \
Baseline                             2.033385          1.884051   
AdversarialDebiase                   1.716931          0.989561   
GridSearchReduction(EO)              1.709511          1.462300   
GridSearchReduction(DP)              1.642589          1.335284   
Reweight                             2.016160          1.945116   
DisparateImpactRemover(0.5)          1.898723          1.825781   
DisparateImpactRemover(0.7)          1.797268          1.897508   

Disparate Impact             Cross Validate 5   Average  
Baseline                             1.503438  1.689631  
AdversarialDebiase                   1.257602  1.233387  
GridSearchReduction(EO)              1.233797  1.399637  
GridSearchReduction(DP)              1.148326  1.358064  
Reweight                             1.541119  1.756519  
DisparateImpactRemover(0.5)          1.461285  1.650396  
DisparateImpactRemover(0.7)          1.455883  1.617153

Disparate Parity             Cross Validate 1  Cross Validate 2  \
Baseline                             0.059128          0.047793   
AdversarialDebiase                   0.008656          0.015786   
GridSearchReduction(EO)              0.041501          0.029149   
GridSearchReduction(DP)              0.046452          0.030140   
Reweight                             0.072298          0.056496   
DisparateImpactRemover(0.5)          0.066852          0.041730   
DisparateImpactRemover(0.7)          0.058038          0.043585   

Disparate Parity             Cross Validate 3  Cross Validate 4  \
Baseline                             0.083322          0.074396   
AdversarialDebiase                   0.077075         -0.001159   
GridSearchReduction(EO)              0.064441          0.045113   
GridSearchReduction(DP)              0.057768          0.032418   
Reweight                             0.080991          0.078689   
DisparateImpactRemover(0.5)          0.072464          0.071711   
DisparateImpactRemover(0.7)          0.065023          0.075529   

Disparate Parity             Cross Validate 5   Average  
Baseline                             0.048614  0.062651  
AdversarialDebiase                   0.021527  0.024377  
GridSearchReduction(EO)              0.026484  0.041338  
GridSearchReduction(DP)              0.017353  0.036826  
Reweight                             0.053258  0.068346  
DisparateImpactRemover(0.5)          0.045829  0.059717  
DisparateImpactRemover(0.7)          0.046139  0.057663

Equal Opportuniy             Cross Validate 1  Cross Validate 2  \
Baseline                             0.067797          0.029337   
AdversarialDebiase                  -0.029661         -0.060538   
GridSearchReduction(EO)              0.022599         -0.041232   
GridSearchReduction(DP)              0.001412         -0.021737   
Reweight                             0.103107          0.007505   
DisparateImpactRemover(0.5)          0.079096         -0.010432   
DisparateImpactRemover(0.7)          0.057910         -0.009842   

Equal Opportuniy             Cross Validate 3  Cross Validate 4  \
Baseline                             0.111285          0.141108   
AdversarialDebiase                   0.059636         -0.043579   
GridSearchReduction(EO)              0.057994          0.054446   
GridSearchReduction(DP)              0.049336          0.039056   
Reweight                             0.115614          0.136253   
DisparateImpactRemover(0.5)          0.073817          0.098963   
DisparateImpactRemover(0.7)          0.079489          0.104645   

Equal Opportuniy             Cross Validate 5   Average  
Baseline                            -0.036105  0.062684  
AdversarialDebiase                  -0.064457 -0.027720  
GridSearchReduction(EO)             -0.098174 -0.000874  
GridSearchReduction(DP)             -0.120274 -0.010441  
Reweight                            -0.018234  0.068849  
DisparateImpactRemover(0.5)         -0.044335  0.039422  
DisparateImpactRemover(0.7)         -0.052679  0.035905

Equalized Odds               Cross Validate 1  Cross Validate 2  \
Baseline                             0.005676          0.009240   
AdversarialDebiase                  -0.027007         -0.004956   
GridSearchReduction(EO)             -0.004992          0.003874   
GridSearchReduction(DP)              0.006037         -0.000629   
Reweight                             0.013751          0.024708   
DisparateImpactRemover(0.5)          0.013028          0.011433   
DisparateImpactRemover(0.7)          0.006781          0.011491   

Equalized Odds               Cross Validate 3  Cross Validate 4  \
Baseline                             0.032212          0.017331   
AdversarialDebiase                   0.035167         -0.029862   
GridSearchReduction(EO)              0.023028          0.003750   
GridSearchReduction(DP)              0.016887         -0.008896   
Reweight                             0.027520          0.024302   
DisparateImpactRemover(0.5)          0.028625          0.024302   
DisparateImpactRemover(0.7)          0.020202          0.027905   

Equalized Odds               Cross Validate 5   Average  
Baseline                             0.021796  0.017251  
AdversarialDebiase                   0.003440 -0.004644  
GridSearchReduction(EO)              0.008810  0.006894  
GridSearchReduction(DP)              0.001457  0.002971  
Reweight                             0.022860  0.022628  
DisparateImpactRemover(0.5)          0.021389  0.019755  
DisparateImpactRemover(0.7)          0.022505  0.017777

In [19]:
#Feldman et al., 2015 implementation
def DisparateImpactRemover_10_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=1.0,sensitive_attribute="gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(1.0)",DisparateImpactRemover_10_CV)

Accuracy                     Cross Validate 1  Cross Validate 2  \
Baseline                               0.8590            0.8695   
AdversarialDebiase                     0.8490            0.8560   
GridSearchReduction(EO)                0.8570            0.8630   
GridSearchReduction(DP)                0.8595            0.8680   
Reweight                               0.8575            0.8725   
DisparateImpactRemover(0.5)            0.8590            0.8715   
DisparateImpactRemover(0.7)            0.8595            0.8735   
DisparateImpactRemover(1.0)            0.8590            0.8735   

Accuracy                     Cross Validate 3  Cross Validate 4  \
Baseline                               0.8555            0.8730   
AdversarialDebiase                     0.8475            0.8675   
GridSearchReduction(EO)                0.8510            0.8655   
GridSearchReduction(DP)                0.8530            0.8710   
Reweight                               0.8585            0.8710   
DisparateImpactRemover(0.5)            0.8565            0.8715   
DisparateImpactRemover(0.7)            0.8510            0.8715   
DisparateImpactRemover(1.0)            0.8570            0.8715   

Accuracy                     Cross Validate 5  Average  
Baseline                               0.8570   0.8628  
AdversarialDebiase                     0.8475   0.8535  
GridSearchReduction(EO)                0.8565   0.8586  
GridSearchReduction(DP)                0.8610   0.8625  
Reweight                               0.8550   0.8629  
DisparateImpactRemover(0.5)            0.8535   0.8624  
DisparateImpactRemover(0.7)            0.8565   0.8624  
DisparateImpactRemover(1.0)            0.8580   0.8638

Disparate Impact             Cross Validate 1  Cross Validate 2  \
Baseline                             1.561193          1.466090   
AdversarialDebiase                   1.074928          1.127915   
GridSearchReduction(EO)              1.356382          1.236195   
GridSearchReduction(DP)              1.408630          1.255491   
Reweight                             1.724320          1.555878   
DisparateImpactRemover(0.5)          1.669753          1.396437   
DisparateImpactRemover(0.7)          1.550854          1.384249   
DisparateImpactRemover(1.0)          1.592769          1.389442   

Disparate Impact             Cross Validate 3  Cross Validate 4  \
Baseline                             2.033385          1.884051   
AdversarialDebiase                   1.716931          0.989561   
GridSearchReduction(EO)              1.709511          1.462300   
GridSearchReduction(DP)              1.642589          1.335284   
Reweight                             2.016160          1.945116   
DisparateImpactRemover(0.5)          1.898723          1.825781   
DisparateImpactRemover(0.7)          1.797268          1.897508   
DisparateImpactRemover(1.0)          1.903773          2.093283   

Disparate Impact             Cross Validate 5   Average  
Baseline                             1.503438  1.689631  
AdversarialDebiase                   1.257602  1.233387  
GridSearchReduction(EO)              1.233797  1.399637  
GridSearchReduction(DP)              1.148326  1.358064  
Reweight                             1.541119  1.756519  
DisparateImpactRemover(0.5)          1.461285  1.650396  
DisparateImpactRemover(0.7)          1.455883  1.617153  
DisparateImpactRemover(1.0)          1.453055  1.686465

Disparate Parity             Cross Validate 1  Cross Validate 2  \
Baseline                             0.059128          0.047793   
AdversarialDebiase                   0.008656          0.015786   
GridSearchReduction(EO)              0.041501          0.029149   
GridSearchReduction(DP)              0.046452          0.030140   
Reweight                             0.072298          0.056496   
DisparateImpactRemover(0.5)          0.066852          0.041730   
DisparateImpactRemover(0.7)          0.058038          0.043585   
DisparateImpactRemover(1.0)          0.060811          0.042761   

Disparate Parity             Cross Validate 3  Cross Validate 4  \
Baseline                             0.083322          0.074396   
AdversarialDebiase                   0.077075         -0.001159   
GridSearchReduction(EO)              0.064441          0.045113   
GridSearchReduction(DP)              0.057768          0.032418   
Reweight                             0.080991          0.078689   
DisparateImpactRemover(0.5)          0.072464          0.071711   
DisparateImpactRemover(0.7)          0.065023          0.075529   
DisparateImpactRemover(1.0)          0.073709          0.082216   

Disparate Parity             Cross Validate 5   Average  
Baseline                             0.048614  0.062651  
AdversarialDebiase                   0.021527  0.024377  
GridSearchReduction(EO)              0.026484  0.041338  
GridSearchReduction(DP)              0.017353  0.036826  
Reweight                             0.053258  0.068346  
DisparateImpactRemover(0.5)          0.045829  0.059717  
DisparateImpactRemover(0.7)          0.046139  0.057663  
DisparateImpactRemover(1.0)          0.044590  0.060818

Equal Opportuniy             Cross Validate 1  Cross Validate 2  \
Baseline                             0.067797          0.029337   
AdversarialDebiase                  -0.029661         -0.060538   
GridSearchReduction(EO)              0.022599         -0.041232   
GridSearchReduction(DP)              0.001412         -0.021737   
Reweight                             0.103107          0.007505   
DisparateImpactRemover(0.5)          0.079096         -0.010432   
DisparateImpactRemover(0.7)          0.057910         -0.009842   
DisparateImpactRemover(1.0)          0.059322          0.011211   

Equal Opportuniy             Cross Validate 3  Cross Validate 4  \
Baseline                             0.111285          0.141108   
AdversarialDebiase                   0.059636         -0.043579   
GridSearchReduction(EO)              0.057994          0.054446   
GridSearchReduction(DP)              0.049336          0.039056   
Reweight                             0.115614          0.136253   
DisparateImpactRemover(0.5)          0.073817          0.098963   
DisparateImpactRemover(0.7)          0.079489          0.104645   
DisparateImpactRemover(1.0)          0.082475          0.138736   

Equal Opportuniy             Cross Validate 5   Average  
Baseline                            -0.036105  0.062684  
AdversarialDebiase                  -0.064457 -0.027720  
GridSearchReduction(EO)             -0.098174 -0.000874  
GridSearchReduction(DP)             -0.120274 -0.010441  
Reweight                            -0.018234  0.068849  
DisparateImpactRemover(0.5)         -0.044335  0.039422  
DisparateImpactRemover(0.7)         -0.052679  0.035905  
DisparateImpactRemover(1.0)         -0.031990  0.051951

Equalized Odds               Cross Validate 1  Cross Validate 2  \
Baseline                             0.005676          0.009240   
AdversarialDebiase                  -0.027007         -0.004956   
GridSearchReduction(EO)             -0.004992          0.003874   
GridSearchReduction(DP)              0.006037         -0.000629   
Reweight                             0.013751          0.024708   
DisparateImpactRemover(0.5)          0.013028          0.011433   
DisparateImpactRemover(0.7)          0.006781          0.011491   
DisparateImpactRemover(1.0)          0.010457          0.005565   

Equalized Odds               Cross Validate 3  Cross Validate 4  \
Baseline                             0.032212          0.017331   
AdversarialDebiase                   0.035167         -0.029862   
GridSearchReduction(EO)              0.023028          0.003750   
GridSearchReduction(DP)              0.016887         -0.008896   
Reweight                             0.027520          0.024302   
DisparateImpactRemover(0.5)          0.028625          0.024302   
DisparateImpactRemover(0.7)          0.020202          0.027905   
DisparateImpactRemover(1.0)          0.027520          0.029201   

Equalized Odds               Cross Validate 5   Average  
Baseline                             0.021796  0.017251  
AdversarialDebiase                   0.003440 -0.004644  
GridSearchReduction(EO)              0.008810  0.006894  
GridSearchReduction(DP)              0.001457  0.002971  
Reweight                             0.022860  0.022628  
DisparateImpactRemover(0.5)          0.021389  0.019755  
DisparateImpactRemover(0.7)          0.022505  0.017777  
DisparateImpactRemover(1.0)          0.015152  0.017579

In [20]:
#Pleiss et al., 2017 implementation
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing

def CalibratedEqOddsPostprocessing_CV(train_set,test_set):
    #Training
    ceop_model=RandomForestClassifier()
    ceop_model.fit(train_set.features,train_set.labels.ravel())

    train_eop_set=train_set.copy(deepcopy=True)
    train_eop_set.scores=ceop_model.predict(train_set.features)

    ceop=CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
    ceop.fit(train_set,train_eop_set)
    #Testing
    pred=ceop_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=ceop.predict(test_pred)
    return test_pred

cross_validation("CalibratedEqOddsPostprocessing",CalibratedEqOddsPostprocessing_CV)

Accuracy                        Cross Validate 1  Cross Validate 2  \
Baseline                                  0.8590            0.8695   
AdversarialDebiase                        0.8490            0.8560   
GridSearchReduction(EO)                   0.8570            0.8630   
GridSearchReduction(DP)                   0.8595            0.8680   
Reweight                                  0.8575            0.8725   
DisparateImpactRemover(0.5)               0.8590            0.8715   
DisparateImpactRemover(0.7)               0.8595            0.8735   
DisparateImpactRemover(1.0)               0.8590            0.8735   
CalibratedEqOddsPostprocessing            0.8600            0.8695   

Accuracy                        Cross Validate 3  Cross Validate 4  \
Baseline                                  0.8555            0.8730   
AdversarialDebiase                        0.8475            0.8675   
GridSearchReduction(EO)                   0.8510            0.8655   
GridSearchReduction(DP)                   0.8530            0.8710   
Reweight                                  0.8585            0.8710   
DisparateImpactRemover(0.5)               0.8565            0.8715   
DisparateImpactRemover(0.7)               0.8510            0.8715   
DisparateImpactRemover(1.0)               0.8570            0.8715   
CalibratedEqOddsPostprocessing            0.8555            0.8680   

Accuracy                        Cross Validate 5  Average  
Baseline                                  0.8570   0.8628  
AdversarialDebiase                        0.8475   0.8535  
GridSearchReduction(EO)                   0.8565   0.8586  
GridSearchReduction(DP)                   0.8610   0.8625  
Reweight                                  0.8550   0.8629  
DisparateImpactRemover(0.5)               0.8535   0.8624  
DisparateImpactRemover(0.7)               0.8565   0.8624  
DisparateImpactRemover(1.0)               0.8580   0.8638  
CalibratedEqOddsPostprocessing            0.8555   0.8617

Disparate Impact                Cross Validate 1  Cross Validate 2  \
Baseline                                1.561193          1.466090   
AdversarialDebiase                      1.074928          1.127915   
GridSearchReduction(EO)                 1.356382          1.236195   
GridSearchReduction(DP)                 1.408630          1.255491   
Reweight                                1.724320          1.555878   
DisparateImpactRemover(0.5)             1.669753          1.396437   
DisparateImpactRemover(0.7)             1.550854          1.384249   
DisparateImpactRemover(1.0)             1.592769          1.389442   
CalibratedEqOddsPostprocessing          1.706235          1.499876   

Disparate Impact                Cross Validate 3  Cross Validate 4  \
Baseline                                2.033385          1.884051   
AdversarialDebiase                      1.716931          0.989561   
GridSearchReduction(EO)                 1.709511          1.462300   
GridSearchReduction(DP)                 1.642589          1.335284   
Reweight                                2.016160          1.945116   
DisparateImpactRemover(0.5)             1.898723          1.825781   
DisparateImpactRemover(0.7)             1.797268          1.897508   
DisparateImpactRemover(1.0)             1.903773          2.093283   
CalibratedEqOddsPostprocessing          2.114510          1.904459   

Disparate Impact                Cross Validate 5   Average  
Baseline                                1.503438  1.689631  
AdversarialDebiase                      1.257602  1.233387  
GridSearchReduction(EO)                 1.233797  1.399637  
GridSearchReduction(DP)                 1.148326  1.358064  
Reweight                                1.541119  1.756519  
DisparateImpactRemover(0.5)             1.461285  1.650396  
DisparateImpactRemover(0.7)             1.455883  1.617153  
DisparateImpactRemover(1.0)             1.453055  1.686465  
CalibratedEqOddsPostprocessing          1.486079  1.742232

Disparate Parity                Cross Validate 1  Cross Validate 2  \
Baseline                                0.059128          0.047793   
AdversarialDebiase                      0.008656          0.015786   
GridSearchReduction(EO)                 0.041501          0.029149   
GridSearchReduction(DP)                 0.046452          0.030140   
Reweight                                0.072298          0.056496   
DisparateImpactRemover(0.5)             0.066852          0.041730   
DisparateImpactRemover(0.7)             0.058038          0.043585   
DisparateImpactRemover(1.0)             0.060811          0.042761   
CalibratedEqOddsPostprocessing          0.068535          0.053072   

Disparate Parity                Cross Validate 3  Cross Validate 4  \
Baseline                                0.083322          0.074396   
AdversarialDebiase                      0.077075         -0.001159   
GridSearchReduction(EO)                 0.064441          0.045113   
GridSearchReduction(DP)                 0.057768          0.032418   
Reweight                                0.080991          0.078689   
DisparateImpactRemover(0.5)             0.072464          0.071711   
DisparateImpactRemover(0.7)             0.065023          0.075529   
DisparateImpactRemover(1.0)             0.073709          0.082216   
CalibratedEqOddsPostprocessing          0.084699          0.073685   

Disparate Parity                Cross Validate 5   Average  
Baseline                                0.048614  0.062651  
AdversarialDebiase                      0.021527  0.024377  
GridSearchReduction(EO)                 0.026484  0.041338  
GridSearchReduction(DP)                 0.017353  0.036826  
Reweight                                0.053258  0.068346  
DisparateImpactRemover(0.5)             0.045829  0.059717  
DisparateImpactRemover(0.7)             0.046139  0.057663  
DisparateImpactRemover(1.0)             0.044590  0.060818  
CalibratedEqOddsPostprocessing          0.047841  0.065566

Equal Opportuniy                Cross Validate 1  Cross Validate 2  \
Baseline                                0.067797          0.029337   
AdversarialDebiase                     -0.029661         -0.060538   
GridSearchReduction(EO)                 0.022599         -0.041232   
GridSearchReduction(DP)                 0.001412         -0.021737   
Reweight                                0.103107          0.007505   
DisparateImpactRemover(0.5)             0.079096         -0.010432   
DisparateImpactRemover(0.7)             0.057910         -0.009842   
DisparateImpactRemover(1.0)             0.059322          0.011211   
CalibratedEqOddsPostprocessing          0.064972          0.027000   

Equal Opportuniy                Cross Validate 3  Cross Validate 4  \
Baseline                                0.111285          0.141108   
AdversarialDebiase                      0.059636         -0.043579   
GridSearchReduction(EO)                 0.057994          0.054446   
GridSearchReduction(DP)                 0.049336          0.039056   
Reweight                                0.115614          0.136253   
DisparateImpactRemover(0.5)             0.073817          0.098963   
DisparateImpactRemover(0.7)             0.079489          0.104645   
DisparateImpactRemover(1.0)             0.082475          0.138736   
CalibratedEqOddsPostprocessing          0.114122          0.133881   

Equal Opportuniy                Cross Validate 5   Average  
Baseline                               -0.036105  0.062684  
AdversarialDebiase                     -0.064457 -0.027720  
GridSearchReduction(EO)                -0.098174 -0.000874  
GridSearchReduction(DP)                -0.120274 -0.010441  
Reweight                               -0.018234  0.068849  
DisparateImpactRemover(0.5)            -0.044335  0.039422  
DisparateImpactRemover(0.7)            -0.052679  0.035905  
DisparateImpactRemover(1.0)            -0.031990  0.051951  
CalibratedEqOddsPostprocessing         -0.036105  0.060774

Equalized Odds                  Cross Validate 1  Cross Validate 2  \
Baseline                                0.005676          0.009240   
AdversarialDebiase                     -0.027007         -0.004956   
GridSearchReduction(EO)                -0.004992          0.003874   
GridSearchReduction(DP)                 0.006037         -0.000629   
Reweight                                0.013751          0.024708   
DisparateImpactRemover(0.5)             0.013028          0.011433   
DisparateImpactRemover(0.7)             0.006781          0.011491   
DisparateImpactRemover(1.0)             0.010457          0.005565   
CalibratedEqOddsPostprocessing          0.018914          0.015551   

Equalized Odds                  Cross Validate 3  Cross Validate 4  \
Baseline                                0.032212          0.017331   
AdversarialDebiase                      0.035167         -0.029862   
GridSearchReduction(EO)                 0.023028          0.003750   
GridSearchReduction(DP)                 0.016887         -0.008896   
Reweight                                0.027520          0.024302   
DisparateImpactRemover(0.5)             0.028625          0.024302   
DisparateImpactRemover(0.7)             0.020202          0.027905   
DisparateImpactRemover(1.0)             0.027520          0.029201   
CalibratedEqOddsPostprocessing          0.034078          0.020285   

Equalized Odds                  Cross Validate 5   Average  
Baseline                                0.021796  0.017251  
AdversarialDebiase                      0.003440 -0.004644  
GridSearchReduction(EO)                 0.008810  0.006894  
GridSearchReduction(DP)                 0.001457  0.002971  
Reweight                                0.022860  0.022628  
DisparateImpactRemover(0.5)             0.021389  0.019755  
DisparateImpactRemover(0.7)             0.022505  0.017777  
DisparateImpactRemover(1.0)             0.015152  0.017579  
CalibratedEqOddsPostprocessing          0.021035  0.021973

In [21]:
#Kamiran et al., 2012 implementation
from aif360.algorithms.postprocessing import RejectOptionClassification

def RejectOptionClassification_EO_CV(train_set,test_set):
    #Training
    roc_EO_model=RandomForestClassifier()
    roc_EO_model.fit(train_set.features,train_set.labels.ravel())

    train_roc_EO_set=train_set.copy(deepcopy=True)
    train_roc_EO_set.scores=roc_EO_model.predict(train_set.features)

    roc_EO = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    metric_name="Equal opportunity difference")
    roc_EO.fit(train_set, train_roc_EO_set)
    #Testing
    pred=roc_EO_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=roc_EO.predict(test_pred)
    return test_pred

cross_validation("RejectOptionClassification(EO)",RejectOptionClassification_EO_CV)

Accuracy                        Cross Validate 1  Cross Validate 2  \
Baseline                                  0.8590            0.8695   
AdversarialDebiase                        0.8490            0.8560   
GridSearchReduction(EO)                   0.8570            0.8630   
GridSearchReduction(DP)                   0.8595            0.8680   
Reweight                                  0.8575            0.8725   
DisparateImpactRemover(0.5)               0.8590            0.8715   
DisparateImpactRemover(0.7)               0.8595            0.8735   
DisparateImpactRemover(1.0)               0.8590            0.8735   
CalibratedEqOddsPostprocessing            0.8600            0.8695   
RejectOptionClassification(EO)            0.8580            0.8690   

Accuracy                        Cross Validate 3  Cross Validate 4  \
Baseline                                  0.8555            0.8730   
AdversarialDebiase                        0.8475            0.8675   
GridSearchReduction(EO)                   0.8510            0.8655   
GridSearchReduction(DP)                   0.8530            0.8710   
Reweight                                  0.8585            0.8710   
DisparateImpactRemover(0.5)               0.8565            0.8715   
DisparateImpactRemover(0.7)               0.8510            0.8715   
DisparateImpactRemover(1.0)               0.8570            0.8715   
CalibratedEqOddsPostprocessing            0.8555            0.8680   
RejectOptionClassification(EO)            0.8560            0.8730   

Accuracy                        Cross Validate 5  Average  
Baseline                                  0.8570   0.8628  
AdversarialDebiase                        0.8475   0.8535  
GridSearchReduction(EO)                   0.8565   0.8586  
GridSearchReduction(DP)                   0.8610   0.8625  
Reweight                                  0.8550   0.8629  
DisparateImpactRemover(0.5)               0.8535   0.8624  
DisparateImpactRemover(0.7)               0.8565   0.8624  
DisparateImpactRemover(1.0)               0.8580   0.8638  
CalibratedEqOddsPostprocessing            0.8555   0.8617  
RejectOptionClassification(EO)            0.8560   0.8624

Disparate Impact                Cross Validate 1  Cross Validate 2  \
Baseline                                1.561193          1.466090   
AdversarialDebiase                      1.074928          1.127915   
GridSearchReduction(EO)                 1.356382          1.236195   
GridSearchReduction(DP)                 1.408630          1.255491   
Reweight                                1.724320          1.555878   
DisparateImpactRemover(0.5)             1.669753          1.396437   
DisparateImpactRemover(0.7)             1.550854          1.384249   
DisparateImpactRemover(1.0)             1.592769          1.389442   
CalibratedEqOddsPostprocessing          1.706235          1.499876   
RejectOptionClassification(EO)          1.631166          1.470490   

Disparate Impact                Cross Validate 3  Cross Validate 4  \
Baseline                                2.033385          1.884051   
AdversarialDebiase                      1.716931          0.989561   
GridSearchReduction(EO)                 1.709511          1.462300   
GridSearchReduction(DP)                 1.642589          1.335284   
Reweight                                2.016160          1.945116   
DisparateImpactRemover(0.5)             1.898723          1.825781   
DisparateImpactRemover(0.7)             1.797268          1.897508   
DisparateImpactRemover(1.0)             1.903773          2.093283   
CalibratedEqOddsPostprocessing          2.114510          1.904459   
RejectOptionClassification(EO)          1.905382          1.771008   

Disparate Impact                Cross Validate 5   Average  
Baseline                                1.503438  1.689631  
AdversarialDebiase                      1.257602  1.233387  
GridSearchReduction(EO)                 1.233797  1.399637  
GridSearchReduction(DP)                 1.148326  1.358064  
Reweight                                1.541119  1.756519  
DisparateImpactRemover(0.5)             1.461285  1.650396  
DisparateImpactRemover(0.7)             1.455883  1.617153  
DisparateImpactRemover(1.0)             1.453055  1.686465  
CalibratedEqOddsPostprocessing          1.486079  1.742232  
RejectOptionClassification(EO)          1.512580  1.658125

Disparate Parity                Cross Validate 1  Cross Validate 2  \
Baseline                                0.059128          0.047793   
AdversarialDebiase                      0.008656          0.015786   
GridSearchReduction(EO)                 0.041501          0.029149   
GridSearchReduction(DP)                 0.046452          0.030140   
Reweight                                0.072298          0.056496   
DisparateImpactRemover(0.5)             0.066852          0.041730   
DisparateImpactRemover(0.7)             0.058038          0.043585   
DisparateImpactRemover(1.0)             0.060811          0.042761   
CalibratedEqOddsPostprocessing          0.068535          0.053072   
RejectOptionClassification(EO)          0.065333          0.049525   

Disparate Parity                Cross Validate 3  Cross Validate 4  \
Baseline                                0.083322          0.074396   
AdversarialDebiase                      0.077075         -0.001159   
GridSearchReduction(EO)                 0.064441          0.045113   
GridSearchReduction(DP)                 0.057768          0.032418   
Reweight                                0.080991          0.078689   
DisparateImpactRemover(0.5)             0.072464          0.071711   
DisparateImpactRemover(0.7)             0.065023          0.075529   
DisparateImpactRemover(1.0)             0.073709          0.082216   
CalibratedEqOddsPostprocessing          0.084699          0.073685   
RejectOptionClassification(EO)          0.076358          0.069025   

Disparate Parity                Cross Validate 5   Average  
Baseline                                0.048614  0.062651  
AdversarialDebiase                      0.021527  0.024377  
GridSearchReduction(EO)                 0.026484  0.041338  
GridSearchReduction(DP)                 0.017353  0.036826  
Reweight                                0.053258  0.068346  
DisparateImpactRemover(0.5)             0.045829  0.059717  
DisparateImpactRemover(0.7)             0.046139  0.057663  
DisparateImpactRemover(1.0)             0.044590  0.060818  
CalibratedEqOddsPostprocessing          0.047841  0.065566  
RejectOptionClassification(EO)          0.051401  0.062328

Equal Opportuniy                Cross Validate 1  Cross Validate 2  \
Baseline                                0.067797          0.029337   
AdversarialDebiase                     -0.029661         -0.060538   
GridSearchReduction(EO)                 0.022599         -0.041232   
GridSearchReduction(DP)                 0.001412         -0.021737   
Reweight                                0.103107          0.007505   
DisparateImpactRemover(0.5)             0.079096         -0.010432   
DisparateImpactRemover(0.7)             0.057910         -0.009842   
DisparateImpactRemover(1.0)             0.059322          0.011211   
CalibratedEqOddsPostprocessing          0.064972          0.027000   
RejectOptionClassification(EO)          0.097458         -0.005948   

Equal Opportuniy                Cross Validate 3  Cross Validate 4  \
Baseline                                0.111285          0.141108   
AdversarialDebiase                      0.059636         -0.043579   
GridSearchReduction(EO)                 0.057994          0.054446   
GridSearchReduction(DP)                 0.049336          0.039056   
Reweight                                0.115614          0.136253   
DisparateImpactRemover(0.5)             0.073817          0.098963   
DisparateImpactRemover(0.7)             0.079489          0.104645   
DisparateImpactRemover(1.0)             0.082475          0.138736   
CalibratedEqOddsPostprocessing          0.114122          0.133881   
RejectOptionClassification(EO)          0.095462          0.102990   

Equal Opportuniy                Cross Validate 5   Average  
Baseline                               -0.036105  0.062684  
AdversarialDebiase                     -0.064457 -0.027720  
GridSearchReduction(EO)                -0.098174 -0.000874  
GridSearchReduction(DP)                -0.120274 -0.010441  
Reweight                               -0.018234  0.068849  
DisparateImpactRemover(0.5)            -0.044335  0.039422  
DisparateImpactRemover(0.7)            -0.052679  0.035905  
DisparateImpactRemover(1.0)            -0.031990  0.051951  
CalibratedEqOddsPostprocessing         -0.036105  0.060774  
RejectOptionClassification(EO)         -0.019644  0.054064

Equalized Odds                  Cross Validate 1  Cross Validate 2  \
Baseline                                0.005676          0.009240   
AdversarialDebiase                     -0.027007         -0.004956   
GridSearchReduction(EO)                -0.004992          0.003874   
GridSearchReduction(DP)                 0.006037         -0.000629   
Reweight                                0.013751          0.024708   
DisparateImpactRemover(0.5)             0.013028          0.011433   
DisparateImpactRemover(0.7)             0.006781          0.011491   
DisparateImpactRemover(1.0)             0.010457          0.005565   
CalibratedEqOddsPostprocessing          0.018914          0.015551   
RejectOptionClassification(EO)          0.006037          0.020322   

Equalized Odds                  Cross Validate 3  Cross Validate 4  \
Baseline                                0.032212          0.017331   
AdversarialDebiase                      0.035167         -0.029862   
GridSearchReduction(EO)                 0.023028          0.003750   
GridSearchReduction(DP)                 0.016887         -0.008896   
Reweight                                0.027520          0.024302   
DisparateImpactRemover(0.5)             0.028625          0.024302   
DisparateImpactRemover(0.7)             0.020202          0.027905   
DisparateImpactRemover(1.0)             0.027520          0.029201   
CalibratedEqOddsPostprocessing          0.034078          0.020285   
RejectOptionClassification(EO)          0.027104          0.019223   

Equalized Odds                  Cross Validate 5   Average  
Baseline                                0.021796  0.017251  
AdversarialDebiase                      0.003440 -0.004644  
GridSearchReduction(EO)                 0.008810  0.006894  
GridSearchReduction(DP)                 0.001457  0.002971  
Reweight                                0.022860  0.022628  
DisparateImpactRemover(0.5)             0.021389  0.019755  
DisparateImpactRemover(0.7)             0.022505  0.017777  
DisparateImpactRemover(1.0)             0.015152  0.017579  
CalibratedEqOddsPostprocessing          0.021035  0.021973  
RejectOptionClassification(EO)          0.020273  0.018592

In [23]:
#Kamiran et al., 2012 implementation
from aif360.algorithms.postprocessing import RejectOptionClassification

def RejectOptionClassification_DP_CV(train_set,test_set):
    #Training
    roc_EO_model=RandomForestClassifier()
    roc_EO_model.fit(train_set.features,train_set.labels.ravel())

    train_roc_EO_set=train_set.copy(deepcopy=True)
    train_roc_EO_set.scores=roc_EO_model.predict(train_set.features)

    roc_EO = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    metric_name="Statistical parity difference")
    roc_EO.fit(train_set, train_roc_EO_set)
    #Testing
    pred=roc_EO_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=roc_EO.predict(test_pred)
    return test_pred

cross_validation("RejectOptionClassification(DP)",RejectOptionClassification_DP_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\s

Accuracy                        Cross Validate 1  Cross Validate 2  \
Baseline                                  0.8590            0.8695   
AdversarialDebiase                        0.8490            0.8560   
GridSearchReduction(EO)                   0.8570            0.8630   
GridSearchReduction(DP)                   0.8595            0.8680   
Reweight                                  0.8575            0.8725   
DisparateImpactRemover(0.5)               0.8590            0.8715   
DisparateImpactRemover(0.7)               0.8595            0.8735   
DisparateImpactRemover(1.0)               0.8590            0.8735   
CalibratedEqOddsPostprocessing            0.8600            0.8695   
RejectOptionClassification(EO)            0.8580            0.8690   
RejectOptionClassification(DP)            0.8610            0.8685   

Accuracy                        Cross Validate 3  Cross Validate 4  \
Baseline                                  0.8555            0.8730   
AdversarialDebiase                        0.8475            0.8675   
GridSearchReduction(EO)                   0.8510            0.8655   
GridSearchReduction(DP)                   0.8530            0.8710   
Reweight                                  0.8585            0.8710   
DisparateImpactRemover(0.5)               0.8565            0.8715   
DisparateImpactRemover(0.7)               0.8510            0.8715   
DisparateImpactRemover(1.0)               0.8570            0.8715   
CalibratedEqOddsPostprocessing            0.8555            0.8680   
RejectOptionClassification(EO)            0.8560            0.8730   
RejectOptionClassification(DP)            0.8555            0.8715   

Accuracy                        Cross Validate 5  Average  
Baseline                                  0.8570   0.8628  
AdversarialDebiase                        0.8475   0.8535  
GridSearchReduction(EO)                   0.8565   0.8586  
GridSearchReduction(DP)                   0.8610   0.8625  
Reweight                                  0.8550   0.8629  
DisparateImpactRemover(0.5)               0.8535   0.8624  
DisparateImpactRemover(0.7)               0.8565   0.8624  
DisparateImpactRemover(1.0)               0.8580   0.8638  
CalibratedEqOddsPostprocessing            0.8555   0.8617  
RejectOptionClassification(EO)            0.8560   0.8624  
RejectOptionClassification(DP)            0.8565   0.8626

Disparate Impact                Cross Validate 1  Cross Validate 2  \
Baseline                                1.561193          1.466090   
AdversarialDebiase                      1.074928          1.127915   
GridSearchReduction(EO)                 1.356382          1.236195   
GridSearchReduction(DP)                 1.408630          1.255491   
Reweight                                1.724320          1.555878   
DisparateImpactRemover(0.5)             1.669753          1.396437   
DisparateImpactRemover(0.7)             1.550854          1.384249   
DisparateImpactRemover(1.0)             1.592769          1.389442   
CalibratedEqOddsPostprocessing          1.706235          1.499876   
RejectOptionClassification(EO)          1.631166          1.470490   
RejectOptionClassification(DP)          1.656480          1.523385   

Disparate Impact                Cross Validate 3  Cross Validate 4  \
Baseline                                2.033385          1.884051   
AdversarialDebiase                      1.716931          0.989561   
GridSearchReduction(EO)                 1.709511          1.462300   
GridSearchReduction(DP)                 1.642589          1.335284   
Reweight                                2.016160          1.945116   
DisparateImpactRemover(0.5)             1.898723          1.825781   
DisparateImpactRemover(0.7)             1.797268          1.897508   
DisparateImpactRemover(1.0)             1.903773          2.093283   
CalibratedEqOddsPostprocessing          2.114510          1.904459   
RejectOptionClassification(EO)          1.905382          1.771008   
RejectOptionClassification(DP)          1.935036          1.946163   

Disparate Impact                Cross Validate 5   Average  
Baseline                                1.503438  1.689631  
AdversarialDebiase                      1.257602  1.233387  
GridSearchReduction(EO)                 1.233797  1.399637  
GridSearchReduction(DP)                 1.148326  1.358064  
Reweight                                1.541119  1.756519  
DisparateImpactRemover(0.5)             1.461285  1.650396  
DisparateImpactRemover(0.7)             1.455883  1.617153  
DisparateImpactRemover(1.0)             1.453055  1.686465  
CalibratedEqOddsPostprocessing          1.486079  1.742232  
RejectOptionClassification(EO)          1.512580  1.658125  
RejectOptionClassification(DP)          1.474471  1.707107

Disparate Parity                Cross Validate 1  Cross Validate 2  \
Baseline                                0.059128          0.047793   
AdversarialDebiase                      0.008656          0.015786   
GridSearchReduction(EO)                 0.041501          0.029149   
GridSearchReduction(DP)                 0.046452          0.030140   
Reweight                                0.072298          0.056496   
DisparateImpactRemover(0.5)             0.066852          0.041730   
DisparateImpactRemover(0.7)             0.058038          0.043585   
DisparateImpactRemover(1.0)             0.060811          0.042761   
CalibratedEqOddsPostprocessing          0.068535          0.053072   
RejectOptionClassification(EO)          0.065333          0.049525   
RejectOptionClassification(DP)          0.067347          0.055093   

Disparate Parity                Cross Validate 3  Cross Validate 4  \
Baseline                                0.083322          0.074396   
AdversarialDebiase                      0.077075         -0.001159   
GridSearchReduction(EO)                 0.064441          0.045113   
GridSearchReduction(DP)                 0.057768          0.032418   
Reweight                                0.080991          0.078689   
DisparateImpactRemover(0.5)             0.072464          0.071711   
DisparateImpactRemover(0.7)             0.065023          0.075529   
DisparateImpactRemover(1.0)             0.073709          0.082216   
CalibratedEqOddsPostprocessing          0.084699          0.073685   
RejectOptionClassification(EO)          0.076358          0.069025   
RejectOptionClassification(DP)          0.077125          0.077082   

Disparate Parity                Cross Validate 5   Average  
Baseline                                0.048614  0.062651  
AdversarialDebiase                      0.021527  0.024377  
GridSearchReduction(EO)                 0.026484  0.041338  
GridSearchReduction(DP)                 0.017353  0.036826  
Reweight                                0.053258  0.068346  
DisparateImpactRemover(0.5)             0.045829  0.059717  
DisparateImpactRemover(0.7)             0.046139  0.057663  
DisparateImpactRemover(1.0)             0.044590  0.060818  
CalibratedEqOddsPostprocessing          0.047841  0.065566  
RejectOptionClassification(EO)          0.051401  0.062328  
RejectOptionClassification(DP)          0.048460  0.065022

Equal Opportuniy                Cross Validate 1  Cross Validate 2  \
Baseline                                0.067797          0.029337   
AdversarialDebiase                     -0.029661         -0.060538   
GridSearchReduction(EO)                 0.022599         -0.041232   
GridSearchReduction(DP)                 0.001412         -0.021737   
Reweight                                0.103107          0.007505   
DisparateImpactRemover(0.5)             0.079096         -0.010432   
DisparateImpactRemover(0.7)             0.057910         -0.009842   
DisparateImpactRemover(1.0)             0.059322          0.011211   
CalibratedEqOddsPostprocessing          0.064972          0.027000   
RejectOptionClassification(EO)          0.097458         -0.005948   
RejectOptionClassification(DP)          0.070621          0.012768   

Equal Opportuniy                Cross Validate 3  Cross Validate 4  \
Baseline                                0.111285          0.141108   
AdversarialDebiase                      0.059636         -0.043579   
GridSearchReduction(EO)                 0.057994          0.054446   
GridSearchReduction(DP)                 0.049336          0.039056   
Reweight                                0.115614          0.136253   
DisparateImpactRemover(0.5)             0.073817          0.098963   
DisparateImpactRemover(0.7)             0.079489          0.104645   
DisparateImpactRemover(1.0)             0.082475          0.138736   
CalibratedEqOddsPostprocessing          0.114122          0.133881   
RejectOptionClassification(EO)          0.095462          0.102990   
RejectOptionClassification(DP)          0.106956          0.116008   

Equal Opportuniy                Cross Validate 5   Average  
Baseline                               -0.036105  0.062684  
AdversarialDebiase                     -0.064457 -0.027720  
GridSearchReduction(EO)                -0.098174 -0.000874  
GridSearchReduction(DP)                -0.120274 -0.010441  
Reweight                               -0.018234  0.068849  
DisparateImpactRemover(0.5)            -0.044335  0.039422  
DisparateImpactRemover(0.7)            -0.052679  0.035905  
DisparateImpactRemover(1.0)            -0.031990  0.051951  
CalibratedEqOddsPostprocessing         -0.036105  0.060774  
RejectOptionClassification(EO)         -0.019644  0.054064  
RejectOptionClassification(DP)         -0.015529  0.058165

Equalized Odds                  Cross Validate 1  Cross Validate 2  \
Baseline                                0.005676          0.009240   
AdversarialDebiase                     -0.027007         -0.004956   
GridSearchReduction(EO)                -0.004992          0.003874   
GridSearchReduction(DP)                 0.006037         -0.000629   
Reweight                                0.013751          0.024708   
DisparateImpactRemover(0.5)             0.013028          0.011433   
DisparateImpactRemover(0.7)             0.006781          0.011491   
DisparateImpactRemover(1.0)             0.010457          0.005565   
CalibratedEqOddsPostprocessing          0.018914          0.015551   
RejectOptionClassification(EO)          0.006037          0.020322   
RejectOptionClassification(DP)          0.014494          0.022188   

Equalized Odds                  Cross Validate 3  Cross Validate 4  \
Baseline                                0.032212          0.017331   
AdversarialDebiase                      0.035167         -0.029862   
GridSearchReduction(EO)                 0.023028          0.003750   
GridSearchReduction(DP)                 0.016887         -0.008896   
Reweight                                0.027520          0.024302   
DisparateImpactRemover(0.5)             0.028625          0.024302   
DisparateImpactRemover(0.7)             0.020202          0.027905   
DisparateImpactRemover(1.0)             0.027520          0.029201   
CalibratedEqOddsPostprocessing          0.034078          0.020285   
RejectOptionClassification(EO)          0.027104          0.019223   
RejectOptionClassification(DP)          0.025655          0.027490   

Equalized Odds                  Cross Validate 5   Average  
Baseline                                0.021796  0.017251  
AdversarialDebiase                      0.003440 -0.004644  
GridSearchReduction(EO)                 0.008810  0.006894  
GridSearchReduction(DP)                 0.001457  0.002971  
Reweight                                0.022860  0.022628  
DisparateImpactRemover(0.5)             0.021389  0.019755  
DisparateImpactRemover(0.7)             0.022505  0.017777  
DisparateImpactRemover(1.0)             0.015152  0.017579  
CalibratedEqOddsPostprocessing          0.021035  0.021973  
RejectOptionClassification(EO)          0.020273  0.018592  
RejectOptionClassification(DP)          0.015100  0.020985

In [24]:
Accuracy_data.to_csv("Benchmark_churn_accuracy.csv")
DisparateImpact_data.to_csv("Benchmark_churn_disparate_impact.csv")
DisparateParity_data.to_csv("Benchmark_churn_disparate_parity.csv")
EqualOpportunity_data.to_csv("Benchmark_churn_equal_opportunity.csv")
EqualizedOdds_data.to_csv("Benchmark_churn_equalized_odds.csv")